To do:
- ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - In addition to this, look at the implications of gradient accumulation steps
    - Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament



In [ ]:
%pip install -r "requirements_bert.txt"

In [14]:
import torch
import transformers
import bitsandbytes
import accelerate
import datasets
import numpy as np
import pandas as pd
import keras
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
from keras.losses import binary_crossentropy
from sklearn.metrics import accuracy_score, f1_score, average_precision_score

In [14]:
del model
torch.cuda.empty_cache()

In [2]:
model_name = "jhu-clsp/mmBERT-base"

quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True,
                                         )

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
lora_config = LoraConfig(
    r=16,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear",  # Fine-tuning all linear (classification, attention... layers)
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,416,096 || all params: 310,947,874 || trainable%: 1.0986


In [39]:
'''
Consider the batch size, could be increased for efficiency purposes.
ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
Have a look at learning rate and gradient norm clipping which I need to read up on.
    In addition to this, look at the implications of gradient accumulation steps
    Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
Early stopping: load_best_model_at_end=True
'''

training_args = TrainingArguments(
    output_dir='./full_results',
    optim="paged_adamw_8bit",
    learning_rate=2e-4, # Learning rate copied from mmBERT paper (8e-4) as they found this to perform best
    num_train_epochs=3,
    per_device_train_batch_size=256, # Batching at 256 to balance generalization and efficient training
    gradient_accumulation_steps=1,  # Gradient of 1 as full batch fits in memory

    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    bf16=True,  # Enable mixed precision
    fp16=False,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    remove_unused_columns=True, # Avoiding manual handling of residual text columns
    max_grad_norm=1.0,

    disable_tqdm=False,
)

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], 
    padding="max_length", 
    truncation=True,
    max_length=392, # Padding to 392 to massively cut down on computation compared to base 8,192 tokens. 
    )

In [ ]:
def weighted_bincrossentropy(true, pred, weight_zero = 99.0, weight_one = 1):
    """
    Calculates weighted binary cross entropy. The weights are fixed.
        
    This can be useful for unbalanced catagories.
    
    Adjust the weights here depending on what is required.
    
    For example if there are 10x as many positive classes as negative classes,
        if you adjust weight_zero = 1.0, weight_one = 0.1, then false positives 
        will be penalize 10 times as much as false negatives.

    """
  
    # calculate the binary cross entropy
    bin_crossentropy = binary_crossentropy(true, pred)
    
    # apply the weights
    weights = true * weight_one + (1. - true) * weight_zero
    weights /= (weight_one + weight_zero) # Normalizing to be more consistent with regular BCE for comparison 
    weighted_bin_crossentropy = weights * bin_crossentropy 

    return np.mean(weighted_bin_crossentropy)

In [37]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #From logits to probabilities
    probs_2d = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    probs = probs_2d[:, 1]  # positive class extraction
    
    weigthted_bce = weighted_bincrossentropy(labels, probs)
    keras_bce = binary_crossentropy(labels, probs)
    keras_bce = float(np.mean(keras_bce.numpy()))  # Converting from keras eagertensor to float value
    
    # Wrapping all metrics to floats for json serialization during model eval
    return {
        'keras_BCE': keras_bce,
        'weigthed BCE': weigthted_bce,
        'precision': float(average_precision_score(labels, probs)),
        'accuracy': float(accuracy_score(labels, probs.round())), # Need rounding for these two computations (integer required)
        'f1': float(f1_score(labels, probs.round(), average='macro')) # macro f1 is better for imbalanced dataset
    }

In [8]:

dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data_3_4_5_temps.json")


In [26]:
val_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json")

val_dataframe = val_dataframe[['text', 'label']]

val_dataset = Dataset.from_pandas(val_dataframe)

tokenized_val = val_dataset.map(tokenize_function, batched=True, num_proc=8)

Map (num_proc=8): 100%|██████████| 176/176 [00:00<00:00, 251.38 examples/s]


In [30]:
test_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data.json")

test_dataframe = test_dataframe[['text', 'label']]

test_dataframe = test_dataframe[0:10000]

test_dataset = Dataset.from_pandas(test_dataframe)

tokenized_test = test_dataset.map(tokenize_function)

Map: 100%|██████████| 10000/10000 [00:01<00:00, 5280.36 examples/s]


In [31]:
len(test_dataframe)

10000

In [ ]:
#val_dataframe = val_dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

dataframe = dataframe[['text', 'label']]

#val_dataset = Dataset.from_pandas(val_dataframe)

dataset = Dataset.from_pandas(dataframe)

#tokenized_val = val_dataset.map(tokenize_function)

# I suspect num_proc can be increased after having identified the padding problem
tokenized_dataset = dataset.map(tokenize_function, batched=True, num_proc=8)

Map (num_proc=8): 100%|██████████| 398842/398842 [00:18<00:00, 21240.25 examples/s]


In [40]:
'''
Look into learning rates, model is currently overfitting quite drastically ("small" test-set)
Normalizing weigthed BCE or no?
Look into regularization, dropout and early stopping to avoid overfitting
'''

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_test,#dataset,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)


trainer.train()

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Keras Bce,Weigthed bce,Precision,Accuracy,F1
1,0.000000,2.185664,1.922872,1.293350,0.763899,0.823864,0.789628
2,0.000000,1.989829,1.773121,1.192626,0.715385,0.823864,0.797978
3,0.000000,2.015357,1.814601,1.220526,0.708619,0.823864,0.794012


/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  ret

TrainOutput(global_step=120, training_loss=0.0010901039194626113, metrics={'train_runtime': 413.5878, 'train_samples_per_second': 72.536, 'train_steps_per_second': 0.29, 'total_flos': 8067821509440000.0, 'train_loss': 0.0010901039194626113, 'epoch': 3.0})

In [41]:
FINE_TUNED_MODEL_NAME = "mmBlameBERT-pol-DA"

merged_model = model.merge_and_unload()    # PEFT: incorporates LoRA into base weights
merged_dir = f"./{FINE_TUNED_MODEL_NAME}-merged"
merged_model.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)
print("✓ Merged model saved to:", merged_dir)

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


✓ Merged model saved to: ./mmBlameBERT-pol-DA-merged


In [ ]:
# Configuration
LORA_MODEL_PATH = "/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/output/mmBERT/template_3_4_5_model"
BASE_MODEL = "jhu-clsp/mmBERT-base"  # Update if different
OUTPUT_PATH = "/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/output/mmBERT/template_3_4_5_merged"
NUM_LABELS = 2  # Binary classification

def merge_and_save_model(lora_path, base_model_name, output_path, num_labels=2):
    """
    Merge LoRA adapters with base model and save as a standard model.
    This creates a standalone model that can be loaded directly without PEFT.
    """
    print("="*70)
    print("Merging LoRA Adapters with Base Model")
    print("="*70)
    
    # Load tokenizer
    print(f"\n1. Loading tokenizer from {base_model_name}...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Load base model with classification head
    print(f"2. Loading base model: {base_model_name}")
    base_model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        num_labels=num_labels,
        trust_remote_code=True
    )
    
    # Load LoRA adapters
    print(f"3. Loading LoRA adapters from {lora_path}...")
    model_with_lora = PeftModel.from_pretrained(base_model, lora_path)
    
    # Merge LoRA weights into base model
    print("4. Merging LoRA weights with base model...")
    merged_model = model_with_lora.merge_and_unload()
    
    # Save merged model
    print(f"5. Saving merged model to {output_path}...")
    merged_model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    
    print("\n" + "="*70)
    print("✓ Model successfully merged and saved!")
    print("="*70)
    print(f"\nYou can now load it directly with:")
    print(f"  model = AutoModelForSequenceClassification.from_pretrained('{output_path}')")
    print(f"  tokenizer = AutoTokenizer.from_pretrained('{output_path}')")
    print("\nNo PEFT library needed for inference!")


# Merge and save the model
merge_and_save_model(
    lora_path=LORA_MODEL_PATH,
    base_model_name=BASE_MODEL,
    output_path=OUTPUT_PATH,
    num_labels=NUM_LABELS
)


Actual batch size: 20


In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:

with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/EvalResultFullData.txt", "w") as f:
    f.write(str(eval_results))

In [ ]:
'''
We would expect to see a gradual decrease in both training and validation loss.
If either om them split too far from eachother that indicates issues with the training process.
The process itself should be pretty smooth with no dips either up or down.
'''

In [ ]:
# This is where we should very much remember to save the finetuned model locally as this contains the new weights for use in analyzing new text
model.save_pretrained(f"output/mmBERT/full_model")

In [ ]:
# OG function without bce
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return {
        'precision': average_precision_score(labels, predictions),
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro') # Macro is better suited for imbalanced data
    }

In [ ]:
import time
import torch

# Profile training speed
model.train()
dummy_batch_sizes = [16, 32, 64, 128]

for bs in dummy_batch_sizes:
    # Create dummy batch
    dummy_input = {
        'input_ids': torch.randint(0, 30000, (bs, 128)).cuda(),
        'attention_mask': torch.ones(bs, 128).cuda(),
        'labels': torch.randint(0, model.config.num_labels, (bs,)).cuda(),
    }
    
    # Warmup
    for _ in range(5):
        try:
            outputs = model(**dummy_input)
            loss = outputs.loss
            loss.backward()
        except RuntimeError as e:
            print(f"Batch size {bs}: OOM")
            break
    
    # Time it
    torch.cuda.synchronize()
    start = time.time()
    for _ in range(20):
        outputs = model(**dummy_input)
        loss = outputs.loss
        loss.backward()
    torch.cuda.synchronize()
    elapsed = time.time() - start
    
    samples_per_sec = (bs * 20) / elapsed
    hours_for_400k = 400000 / samples_per_sec / 3600
    
    print(f"Batch size {bs}: {samples_per_sec:.1f} samples/sec, Est. time: {hours_for_400k:.1f}h")